In [10]:
import sys
sys.path[0]+='/..'
from dataset import GraphDataset
import torch
import torch_geometric
import os 
from model.gcn import GNNPolicy
import time
from torch.optim.lr_scheduler import CosineAnnealingLR

In [11]:
#set folder

model_save_path = '../model_save/3bin_10_pretrain_routine/'
log_save_path = "../model_save/3bin_10_log_routine/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [12]:
LEARNING_RATE = 1e-4
NB_EPOCHS =2000
BATCH_SIZE = 64
NUM_WORKERS = 2
ETA_MIN=1e-5

In [13]:
train_data = GraphDataset('../datasets/10_std/3bin/train/')
valid_data = GraphDataset('../datasets/10_std/3bin/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [14]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [15]:
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")


载入模型成功


In [16]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
            )
            # compute loss
            
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            loss = loss_fn(presol, weighted_sol)
            
            

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [17]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS, eta_min=ETA_MIN)
best_val_loss = 0.2

In [18]:
cur_epoch = 0
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f}")
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print(best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>300 and valid_loss>best_val_loss:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.062185	Epoch 0 Valid loss: 0.054810
0.054810453206300735
Epoch 1 Train loss: 0.049331	Epoch 1 Valid loss: 0.058460
Epoch 2 Train loss: 0.048629	Epoch 2 Valid loss: 0.053789
0.053789264832933746
Epoch 3 Train loss: 0.049048	Epoch 3 Valid loss: 0.055780
Epoch 4 Train loss: 0.048762	Epoch 4 Valid loss: 0.055034
Epoch 5 Train loss: 0.049833	Epoch 5 Valid loss: 0.056732
Epoch 6 Train loss: 0.048948	Epoch 6 Valid loss: 0.055589
Epoch 7 Train loss: 0.050149	Epoch 7 Valid loss: 0.054969
Epoch 8 Train loss: 0.048332	Epoch 8 Valid loss: 0.054305
Epoch 9 Train loss: 0.047975	Epoch 9 Valid loss: 0.054143
Epoch 10 Train loss: 0.048961	Epoch 10 Valid loss: 0.055911
Epoch 11 Train loss: 0.048520	Epoch 11 Valid loss: 0.057174
Epoch 12 Train loss: 0.049080	Epoch 12 Valid loss: 0.056564
Epoch 13 Train loss: 0.050180	Epoch 13 Valid loss: 0.056611
Epoch 14 Train loss: 0.049923	Epoch 14 Valid loss: 0.055062
Epoch 15 Train loss: 0.048740	Epoch 15 Valid loss: 0.058256
Epoch 16 Train los